In [ ]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns

# 1. Load the raw VCF and start initial filtering

### --- Skip to (3) if you don't need work with the raw file ---

In [ ]:
vcf = pd.read_csv("IBM_DP10_AD_ShortNames.vcf.txt", delimiter="\t",skiprows=301)
vcf = vcf[vcf["#CHROM"].isin([1,2,3,4,5,6,7,8,9,10])]
vcf = vcf.drop_duplicates().reset_index(drop=True)
vcf

# 2. Function to modify the vcf IDs to gene IDs based on coordinates

In [ ]:
# The VCF was generated by calling bcftools mpileup
geneIDs = pd.read_csv("gene_ids_by_region.tsv", delimiter="\t", header=None)

In [ ]:
last_chrom = vcf["#CHROM"].iloc[0]   # keep track of the last chromosome name use, first one is 1
geneID_list = [] # a list to keep all the gene IDs based on vcf coordinates
count=0          # for each chromosome use the count to iterate over the rows

# Trying to speed it up a bit by keeping chromosomes separate
vcf_chrom = vcf[vcf["#CHROM"]==last_chrom]     
geneIDs_chrom = geneIDs[geneIDs[0]==last_chrom] # for both the vcf and bed-like file

for v in range(len(vcf)):
    current_chrom = vcf["#CHROM"].iloc[v]
    if current_chrom != last_chrom:
        vcf_chrom = vcf[vcf["#CHROM"] == current_chrom] # change temp files to current chromosome
        geneIDs_chrom = geneIDs[geneIDs[0] == current_chrom]        # for both the vcf and bed-like file
        count = 0 # restart the count to iterate over temp files
    
    # Finds all geneIDs with coordinates sorrounding SNP coordinates, and keeps only the first gene ID
    vcf_pos = vcf_chrom["POS"].iloc[count]
    geneID_list.append(geneIDs_chrom[
        (geneIDs_chrom[1] <= vcf_pos) &
        (geneIDs_chrom[2] >= vcf_pos)].iloc[0,3])
    last_chrom = vcf["#CHROM"].iloc[v]
    count += 1
print(len(vcf), len(geneID_list))

In [ ]:
vcf["ID"] = geneID_list # Change the ID to to the gene names
# sort the vcf file based on chromosome and position of SNPs
vcf = vcf.sort_values(['#CHROM', 'POS'], ascending=[True, True]).reset_index().drop("index", axis=1)
vcf.to_csv("IBM_DP10_AD_ShortNames_geneIDs.vcf", index=False, sep="\t", quoting=csv.QUOTE_NONE)
# Manually added back the original 300 VCF header lines 

# 3. Load the raw VCF file containing chromsomes 1-10 and gene IDs

In [ ]:
# Short names is the file where I converted the bam file names to the line name manually
vcf = pd.read_csv("IBM_DP10_AD_ShortNames_geneIDs.vcf", delimiter="\t")#,skiprows=301)
vcf = vcf[vcf["QUAL"] == 999.0]
# Keep VCF metadata with the original row index for later access
vcf_coordinates = vcf[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT"]] 
vcf = vcf.drop(["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT"], axis=1)
print(len(vcf))
vcf

# Split the VCF format to multiple files

In [ ]:
def GT(val):
    genotype = val.split(":")[0]
    if genotype == "0/0":
        return "B"
    elif genotype == "1/1":
        return "M"
    else:
        return "H"
    
def PL(val):
    return val.split(":")[1]

def DP(val):
    return int(val.split(":")[2])

def AD(val):
    return val.split(":")[3]

def AD_B73(val):
    return int(val.split(",")[0])
def AD_Mo17(val):
    return int(val.split(",")[1])

In [ ]:
GTdf = vcf.applymap(GT)
PLdf = vcf.applymap(PL)
DPdf = vcf.applymap(DP)
ADdf = vcf.applymap(AD)
Bdf  = ADdf.applymap(AD_B73)
Mdf  = ADdf.applymap(AD_Mo17)
print(len(vcf), len(GTdf), len(PLdf), len(DPdf), len(ADdf), len(Bdf), len(Mdf))

# 4. Apply a minimal DP filter and remove SNPs that don't match parents

In [ ]:
def DP_filter(val):
    # For each value, if read depth < 10 return np.NaN
    if val < 10:
        return np.NaN
    else:
        return val

DPdf = DPdf.applymap(DP_filter)
# thresh=N requires that a column has at least N non-NaNs to survive.
DPdf = DPdf.dropna(thresh = 10, axis = 0) 
print("After filtering:", len(DPdf))

In [ ]:
# Filter sites that don't match the parent inbred lines
GTdf = GTdf[GTdf["B73"]=="B"]
GTdf = GTdf[GTdf["Mo17"]=="M"]
print("After filtering:", len(GTdf))

In [ ]:
# Keep only rows that made it through both previous filtering
intersect = DPdf.index.intersection(GTdf.index)
vcf_coordinates  = vcf_coordinates.loc[intersect]
GTdf = GTdf.loc[intersect]
DPdf = DPdf.loc[intersect]
Bdf  = Bdf.loc[intersect]
Mdf  = Mdf.loc[intersect]
print(len(vcf_coordinates), len(GTdf), len(DPdf), len(Bdf), len(Mdf))

# 5. Filter SNPs based on gene ID groupings

In [ ]:
GTdf2 = GTdf.copy()
GTdf2.index = list(vcf_coordinates["ID"])

In [ ]:
# This function returns the most common genotype as a string (B,M,H,N)
# If two genotypes are equally common it returns a list of them
GTgrouped = GTdf2.groupby(GTdf2.index).agg(pd.Series.mode)
print(len(GTgrouped))

In [ ]:
# Doesn't seem to be necessary but I would keep it just in case some of the filtering parameters
# generate different variations of SNP calls that should probably be ignored if not relatively certain
def list_to_NA(val):
    # if the previous mode function returns a numpy.ndarray list call the allele as np.NaN
    # All alleles should either that list or "B", "M", "H" so it works
    if type(val) != str:
        return np.NaN
    else:
        return val

GTgrouped = GTgrouped.applymap(list_to_NA)
GTgrouped = GTgrouped.dropna(thresh = 10, axis = 0) # if row has more less than thresh np.NaNs then drop it
# There are no column with at least 10 non-NaN values
print(len(GTgrouped))

In [ ]:
vcf_coordinates = vcf_coordinates[vcf_coordinates["ID"].isin(GTgrouped.index)]
GTdf = GTdf.loc[vcf_coordinates.index]
Bdf  = Bdf.loc[vcf_coordinates.index]
Mdf  = Mdf.loc[vcf_coordinates.index]
print(len(vcf_coordinates), len(GTdf), len(Bdf), len(Mdf))

In [ ]:
# The previous step was the last that required exact numerical index
# Now we can add the mapped gene IDs to the ID column and index columns
# Note that this removes the original index mapping
GTdf.index = list(vcf_coordinates["ID"])
Bdf.index  = list(vcf_coordinates["ID"])
Mdf.index  = list(vcf_coordinates["ID"])

In [ ]:
def allele_comparer(row):
    # This function is useful to make the box-plot figure to check the correctness of the grouped genotype file
    # since the grouped genotypes will not always match the the individual SNPs in the filtered SNP genotype file
    # If genotypes match cell returns True and if it doesn't match it returns False
    # In other words - this df specifies which cells were used and not used to call the grouped genotpye
    return(row==GTgrouped.loc[row.name])
compare_GT = GTdf.apply(allele_comparer, axis=1)

In [ ]:
# Reorder it, I am not sure why it was disordered but it makes the next cell break
# As of CPython/PyPy 3.6 (and as a language guarantee in 3.7), plain dict is insertion ordered
# https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order
# Might not work on older python versions but the link has alternative options
GTgrouped = GTgrouped.loc[list(dict.fromkeys(compare_GT.index))]

In [ ]:
# By converting to 0,1 (astype(int)) I can multiple with allele count and keep the count if True
compare_GT = compare_GT.astype(int)
compare_Bdf = compare_GT * Bdf
compare_Mdf = compare_GT * Mdf
print(len(GTdf), len(compare_GT), len(Bdf), len(compare_Mdf))

# 6. QC results by comparing predicted SNPs to reference SNPs

In [ ]:
# Now that I have the correct counts I want to separate them based on called and alternative counts
# This is done to make a box-plot as a quality control proof of concept to compare between all
def rfc(row, rdf, allele):
    # reference count function
    # rdf: reference data frame
    # al: reference allele to check for 
    return(rdf.loc[row.name][row==allele])

In [ ]:
# https://stackoverflow.com/questions/52677165/how-to-pad-pandas-dataframe-column-with-different-sizes-with-nan-values
def pad_dict_list(dict_list, padel=""):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

In [ ]:
FACd = {"B73_B73":  [x for x in fFAC.apply(rfc, rdf=B_counts, allele="B", axis=1).to_numpy().ravel() if not pd.isnull(x)],
        "B73_Mo17": [x for x in fFAC.apply(rfc, rdf=B_counts, allele="M", axis=1).to_numpy().ravel() if not pd.isnull(x)],
        "Mo17_B73":  [x for x in fFAC.apply(rfc, rdf=M_counts, allele="B", axis=1).to_numpy().ravel() if not pd.isnull(x)],
        "Mo17_Mo17":  [x for x in fFAC.apply(rfc, rdf=M_counts, allele="M", axis=1).to_numpy().ravel() if not pd.isnull(x)],
        "Het_B73":  [x for x in fFAC.apply(rfc, rdf=B_counts, allele="H", axis=1).to_numpy().ravel() if not pd.isnull(x)],
        "Het_Mo17":  [x for x in fFAC.apply(rfc, rdf=M_counts, allele="H", axis=1).to_numpy().ravel() if not pd.isnull(x)]}
FACd = pad_dict_list(FACd)
pd.DataFrame(FACd).to_csv("box_plot_test.csv")

# 7. Save the final IBM map as a CSV file

In [ ]:
# Reorder it (As of CPython/PyPy 3.6 (and as a language guarantee in 3.7), plain dict is insertion ordered)
# https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order
GTgrouped2 = GTgrouped.loc[list(dict.fromkeys(compare_GT.index))]
GTgrouped2 = GTgrouped2.fillna("N")

temp = vcf_coordinates.loc[vcf_coordinates.groupby('ID').POS.idxmin()]
temp = temp.sort_values(['#CHROM', 'POS'], ascending=[True, True]).reset_index().drop("index", axis=1)
temp = temp[["ID", "#CHROM", "POS"]]
temp = temp.set_index(["ID"])
temp.index.name = None
temp.join(GTgrouped2).to_csv("210417 - IBM SNP Map.csv")